Imported Libraries

In [94]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
from bs4 import BeautifulSoup
from transformers import pipeline

In [95]:
url = "https://quotes.toscrape.com" #target Website
response = requests.get(url) #Response stores the response from the server

response.status_code #status of response

200

In [96]:
soup = BeautifulSoup(response.text, "html.parser") #html.parser us used to interpret the HTML Data

In [97]:
import requests
from bs4 import BeautifulSoup

url = "https://quotes.toscrape.com"
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

# extract quotes
quotes = soup.find_all("span", class_="text")

for q in quotes[:2]:
    print(q.text)


“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
“It is our choices, Harry, that show what we truly are, far more than our abilities.”


In [98]:
url = "https://quotes.toscrape.com"
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")

quotes = soup.find_all("span", class_="text")
authors = soup.find_all("small", class_="author")

quote_list = []
author_list = []

for q, a in zip(quotes, authors):
    quote_list.append(q.text)
    author_list.append(a.text)

df = pd.DataFrame({
    "quote": quote_list,
    "author": author_list
})

df.head(300)


,quote,author
0,“The world as we have created it is a process ...,Albert Einstein
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling
2,“There are only two ways to live your life. On...,Albert Einstein
3,"“The person, be it gentleman or lady, who has ...",Jane Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe
5,“Try not to become a man of success. Rather be...,Albert Einstein
6,“It is better to be hated for what you are tha...,André Gide
7,"“I have not failed. I've just found 10,000 way...",Thomas A. Edison
8,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt
9,"“A day without sunshine is like, you know, nig...",Steve Martin


In [99]:
all_quotes = []
all_authors = []

for page in range(1, 20):   # pages 1 to 20
    url = f"https://quotes.toscrape.com/page/{page}/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    quotes = soup.find_all("span", class_="text")
    authors = soup.find_all("small", class_="author")

    for q, a in zip(quotes, authors):
        all_quotes.append(q.text)
        all_authors.append(a.text)

df = pd.DataFrame({
    "quote": all_quotes,
    "author": all_authors
})

df.shape


(100, 2)

In [100]:
emotion_classifier = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    return_all_scores=True
)


Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [101]:
def get_top_emotion(text):
    results = emotion_classifier(text)[0]
    top_emotion = max(results, key=lambda x: x["score"])
    return top_emotion["label"]


In [102]:
df["emotion"] = df["quote"].apply(get_top_emotion)

df.head()

,quote,author,emotion
0,“The world as we have created it is a process ...,Albert Einstein,neutral
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,neutral
2,“There are only two ways to live your life. On...,Albert Einstein,neutral
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,disgust
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,disgust


Sentiment Analysis

In [103]:
sentiment_analyzer = pipeline("sentiment-analysis")

df["sentiment"] = df["quote"].apply(
    lambda x: sentiment_analyzer(x)[0]["label"]
)

df.head()


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


,quote,author,emotion,sentiment
0,“The world as we have created it is a process ...,Albert Einstein,neutral,POSITIVE
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling,neutral,POSITIVE
2,“There are only two ways to live your life. On...,Albert Einstein,neutral,POSITIVE
3,"“The person, be it gentleman or lady, who has ...",Jane Austen,disgust,NEGATIVE
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe,disgust,POSITIVE


In [104]:
df.to_csv("Final_Data_with_emotion&Sentiments.csv", index=False)